In [3]:
!pip install NERDA

     |████████████████████████████████| 1.3MB 14.7MB/s 
     |████████████████████████████████| 776.8MB 23kB/s 
     |████████████████████████████████| 1.1MB 42.3MB/s 
     |████████████████████████████████| 901kB 41.8MB/s 
     |████████████████████████████████| 2.9MB 27.9MB/s 
  Created wheel for progressbar: filename=progressbar-2.5-cp37-none-any.whl size=12076 sha256=ef475384b8dbef66eef508966c2d9b18ef7b5b74111caa495a11be3c40f1e8ef
  Stored in directory: /root/.cache/pip/wheels/c0/e9/6b/ea01090205e285175842339aa3b491adeb4015206cda272ff0
Successfully built progressbar
ERROR: torchvision 0.9.1+cu101 has requirement torch==1.8.1, but you'll have torch 1.7.1 which is incompatible.
ERROR: torchtext 0.9.1 has requirement torch==1.8.1, but you'll have torch 1.7.1 which is incompatible.
  Found existing installation: torch 1.8.1+cu101
    Uninstalling torch-1.8.1+cu101:
      Successfully uninstalled torch-1.8.1+cu101


In [4]:
!pip install transformers

In [5]:
# https://www.aitimejournal.com/@akshay.chavan/complete-tutorial-on-named-entity-recognition-ner-using-python-and-keras
# https://www.kaggle.com/mervetncr/turkish-text-classification-fasttext-and-bert
# https://data.mendeley.com/datasets/cdcztymf4k/1
# https://www.kaggle.com/behcetsenturk/shrinked-twnertc-turkish-ner-data-by-kuzgunlar
# https://web.itu.edu.tr/gulsenc/papers/NERsubmittedColing.pdf 
#   Şeker, G. A., & Eryiğit, G. (2012). Initial explorations on using CRFs for Turkish
#   named entity recognition. Proceedings of COLING 2012, 2459-2474.
#4 class :Location, Person, Organization, Misc
#import fasttext
from scipy import stats
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from tqdm import tqdm
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer 
import os, re, csv, math, codecs

# check where you stored the file in order to open it!
df= pd.read_csv(r'TWNERTC_TC_Coarse Grained NER_DomainIndependent_NoiseReduction.DUMP', header=None, 
                names=["Category","Tag","Text"], delimiter='\t')

# comparing number of tags, words if they are not equal then drop.
df["Tag"]=df.Tag.str.split()
df["Text"]=df.Text.str.split()
df["LenTag"]=df['Tag'].apply(len)
df["LenText"]=df['Text'].apply(len)
# whole dataset
df["SameLen"]= np.where(df["LenTag"] == df["LenText"], True, False)
print(df["SameLen"].value_counts())
print(len(df))
print(df["LenTag"].sum(),df["LenText"].sum())
df = df[df.SameLen != False]
# equal dataset
print(df["SameLen"].value_counts())
print(len(df))
print(df["LenTag"].sum(),df["LenText"].sum())
df=df.drop(["LenTag","LenText","SameLen"],axis=1)
pd.set_option('display.max_colwidth', None)
df.head()

True     520376
False      6457
Name: SameLen, dtype: int64
526833
8871708 9061262
True    520376
Name: SameLen, dtype: int64
520376
8742256 8742256


,Category,Tag,Text
0,government,"[B-PERSON, I-PERSON, O, B-LOCATION, O, B-MISC, O]","[Corina, Casanova, ,, İsviçre, Federal, Şansölyesidir, .]"
1,law,"[B-MISC, O, B-ORGANIZATION, I-ORGANIZATION, I-ORGANIZATION, I-ORGANIZATION, O, O, O, O, O, O, O, B-MISC, O, O, O]","[Casanova, ,, İsviçre, Federal, Yüksek, Mahkemesi, eski, Başkanı, ,, Nay, Giusep'in, pratiğinde, bir, avukat, olarak, çalıştı, .]"
2,government,"[B-PERSON, I-PERSON, O, B-MISC, O, O]","[Corina, Casanova, bir, federal, parlementerdir, .]"
3,location,"[B-PERSON, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, B-MISC, O, B-MISC, O, B-LOCATION, O, O, O, B-MISC, O]","[Casanova, ,, Hristiyan, Demokrat, Halk, Partisi, üyesi, üyedidir, ve, altı, dilde, konuşmaktadır, :, Romanş, dili, ,, Almanca, ,, Fransızca, ,, İtalyanca, ,, İngilizce, ve, İspanyolca, .]"
4,government,"[B-ORGANIZATION, I-ORGANIZATION, I-ORGANIZATION, O, B-ORGANIZATION, B-MISC, O, O, O, O]","[İsviçre, Dışişleri, Bakanlığı, ,, İsviçre, federal, yönetiminin, bir, bölümüdür, .]"


In [6]:
split_1 = int(0.8 * len(df))
split_2 = int(0.9 * len(df))
train_df = df[:split_1]
dev_df = df[split_1:split_2]
test_df = df[split_2:]

training={'sentences': train_df["Text"].tolist(),'tags':train_df["Tag"].tolist()}
print(len(training.get('sentences')))
print(type(training))

validation ={'sentences': dev_df["Text"].tolist(),'tags':dev_df["Tag"].tolist()}
print(len(validation.get('sentences')))
print(type(validation))

testing ={'sentences': test_df["Text"].tolist(),'tags':test_df["Tag"].tolist()}
print(len(testing.get('sentences')))
print(type(testing))

416300
<class 'dict'>
52038
<class 'dict'>
52038
<class 'dict'>


In [10]:
dropout = 0.1
# hyperparameters for training
training_hyperparameters = {
'epochs' : 1,
'warmup_steps' : 500,                                                   
'train_batch_size': 13,                                         
'learning_rate': 0.0001
}

tag_scheme = [
'B-PERSON',
'I-PERSON',
'B-ORGANIZATION',
'I-ORGANIZATION',
'B-LOCATION',
'I-LOCATION',
'B-MISC',
'I-MISC'
]


from NERDA.models import NERDA
model = NERDA(
dataset_training = training,
dataset_validation = validation ,
tag_scheme = tag_scheme, 
tag_outside = 'O',
transformer = 'dbmdz/bert-base-turkish-cased',
dropout = dropout,
hyperparameters = training_hyperparameters
) 
print("MODEL CREATED")

Device automatically set to: cuda
MODEL CREATED


In [ ]:
print(type(model))
model.train()

In [14]:
print(type(model))
print("Testing started...")
model.evaluate_performance(testing)

<class 'NERDA.models.NERDA'>
Testing started...


,Level,F1-Score
0,B-PERSON,0.647065
1,I-PERSON,0.685585
2,B-ORGANIZATION,0.568911
3,I-ORGANIZATION,0.574827
4,B-LOCATION,0.751515
5,I-LOCATION,0.603653
6,B-MISC,0.683278
7,I-MISC,0.631968
0,AVG_MICRO,0.679156
0,AVG_MACRO,0.643350


In [61]:
deneme= 'Türkiye Büyük Millet Meclisi'
model.predict_text(deneme)

([['Türkiye', 'Büyük', 'Millet', 'Meclisi']],
 [['B-MISC', 'I-MISC', 'I-MISC', 'I-MISC']])

In [62]:
import pickle

# save the model to disk
filename = 'bertNER.sav'
pickle.dump(model, open(filename, 'wb'))
 
# some time later...

# load the model from disk
loaded_model = pickle.load(open(filename, 'rb'))
loaded_model.predict_text(deneme)

([['Türkiye', 'Büyük', 'Millet', 'Meclisi']],
 [['B-MISC', 'I-MISC', 'I-MISC', 'I-MISC']])

In [68]:
# download ner model to local computer
from google.colab import files
files.download('bertNER.sav') 


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

https://towardsdatascience.com/easy-fine-tuning-of-transformers-for-named-entity-recognition-d72f2b5340e3

https://medium.com/analytics-vidhya/named-entity-recognition-for-turkish-with-bert-f8ec04a31b0